# Importing Libraries

In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, accuracy_score, mean_squared_error,r2_score
from random import random
from math import exp
from random import randrange
from random import seed
from csv import reader

# Data Preprocessing

In [2]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [3]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [4]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
            row[column] = lookup[row[column]]
    return lookup

In [5]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

In [6]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [7]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# BPNN Model

In [8]:
# 1. Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [9]:
# 2. Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [10]:
# 3. Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [11]:
# 4. Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [12]:
# 5. Calculate the derivative of a neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

In [13]:
# 6. Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [14]:
# 7. Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
        neuron['weights'][-1] += l_rate * neuron['delta']

In [15]:
# 8. Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [16]:
# 9. Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [17]:
# 10. Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

In [18]:
# 11. Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [19]:
# 12. Evaluate an algorithm using a cross validation split, calculating accuracy using cross validation
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        mse = mean_squared_error(actual,predicted) 
        print("Confusion Matrix:")
        print(confusion_matrix(actual, predicted))
        print("Classification Report:")
        print(classification_report(actual, predicted))
        print("Accuracy Scores:")
        print(accuracy_score(actual, predicted)*100)
        print("Actual Value:")
        print(actual)
        print("Predicted Value:")
        print(predicted)
        print("Mean Squared Error Value:")
        print(mse)
    return scores

# Loading the Dataset and performing BPNN

In [20]:
# load and prepare data
filename = 'Dialysis1_Clean_Noheader.csv'
dataset = load_csv(filename)

In [21]:
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# evaluate algorithm
n_folds = 3
l_rate = 0.5
n_epoch = 500
n_hidden = 12
n_inputs = 131
n_outputs = 1

In [22]:
network = initialize_network(n_inputs,n_hidden,n_outputs)
for layer in network:
    print(layer)

[{'weights': [0.15543830576878448, 0.9406704582362528, 0.5018680713348584, 0.05884637956445238, 0.3628000629079371, 0.778261271933976, 0.633014026531973, 0.5876234802585674, 0.508328557295123, 0.3179724194676705, 0.8627103643245896, 0.8554730098067824, 0.4026177948293175, 0.8257116805135573, 0.5067904475621559, 0.16693071116623015, 0.9200412289225892, 0.5155869544448389, 0.9504844020067064, 0.8798786858255448, 0.1344766486952621, 0.9019154695785919, 0.33488972628849134, 0.7902485195596622, 0.3360003989301238, 0.4056966686582868, 0.4712207498341231, 0.41035214701721545, 0.6775174495407923, 0.6939882527314004, 0.3486310750485736, 0.077603425061608, 0.9387992599543626, 0.6364081554074575, 0.39847269042404565, 0.6560775123218183, 0.7156003987125943, 0.6438141929563748, 0.5447174186220154, 0.9554819028197538, 0.8612662390884811, 0.2895169823993594, 0.3776229338025927, 0.9868660410285456, 0.6021629864194264, 0.40610632010400816, 0.778391561610188, 0.9470193999831292, 0.03977259266866484, 0.8

In [23]:
#Test training backprop algorithm
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, n_hidden, n_outputs)
train_network(network, dataset, l_rate, n_epoch, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=0.000
>epoch=1, lrate=0.500, error=0.000
>epoch=2, lrate=0.500, error=0.000
>epoch=3, lrate=0.500, error=0.000
>epoch=4, lrate=0.500, error=0.000
>epoch=5, lrate=0.500, error=0.000
>epoch=6, lrate=0.500, error=0.000
>epoch=7, lrate=0.500, error=0.000
>epoch=8, lrate=0.500, error=0.000
>epoch=9, lrate=0.500, error=0.000
>epoch=10, lrate=0.500, error=0.000
>epoch=11, lrate=0.500, error=0.000
>epoch=12, lrate=0.500, error=0.000
>epoch=13, lrate=0.500, error=0.000
>epoch=14, lrate=0.500, error=0.000
>epoch=15, lrate=0.500, error=0.000
>epoch=16, lrate=0.500, error=0.000
>epoch=17, lrate=0.500, error=0.000
>epoch=18, lrate=0.500, error=0.000
>epoch=19, lrate=0.500, error=0.000
>epoch=20, lrate=0.500, error=0.000
>epoch=21, lrate=0.500, error=0.000
>epoch=22, lrate=0.500, error=0.000
>epoch=23, lrate=0.500, error=0.000
>epoch=24, lrate=0.500, error=0.000
>epoch=25, lrate=0.500, error=0.000
>epoch=26, lrate=0.500, error=0.000
>epoch=27, lrate=0.500, error=0.000
>e

>epoch=227, lrate=0.500, error=0.000
>epoch=228, lrate=0.500, error=0.000
>epoch=229, lrate=0.500, error=0.000
>epoch=230, lrate=0.500, error=0.000
>epoch=231, lrate=0.500, error=0.000
>epoch=232, lrate=0.500, error=0.000
>epoch=233, lrate=0.500, error=0.000
>epoch=234, lrate=0.500, error=0.000
>epoch=235, lrate=0.500, error=0.000
>epoch=236, lrate=0.500, error=0.000
>epoch=237, lrate=0.500, error=0.000
>epoch=238, lrate=0.500, error=0.000
>epoch=239, lrate=0.500, error=0.000
>epoch=240, lrate=0.500, error=0.000
>epoch=241, lrate=0.500, error=0.000
>epoch=242, lrate=0.500, error=0.000
>epoch=243, lrate=0.500, error=0.000
>epoch=244, lrate=0.500, error=0.000
>epoch=245, lrate=0.500, error=0.000
>epoch=246, lrate=0.500, error=0.000
>epoch=247, lrate=0.500, error=0.000
>epoch=248, lrate=0.500, error=0.000
>epoch=249, lrate=0.500, error=0.000
>epoch=250, lrate=0.500, error=0.000
>epoch=251, lrate=0.500, error=0.000
>epoch=252, lrate=0.500, error=0.000
>epoch=253, lrate=0.500, error=0.000
>

>epoch=450, lrate=0.500, error=0.000
>epoch=451, lrate=0.500, error=0.000
>epoch=452, lrate=0.500, error=0.000
>epoch=453, lrate=0.500, error=0.000
>epoch=454, lrate=0.500, error=0.000
>epoch=455, lrate=0.500, error=0.000
>epoch=456, lrate=0.500, error=0.000
>epoch=457, lrate=0.500, error=0.000
>epoch=458, lrate=0.500, error=0.000
>epoch=459, lrate=0.500, error=0.000
>epoch=460, lrate=0.500, error=0.000
>epoch=461, lrate=0.500, error=0.000
>epoch=462, lrate=0.500, error=0.000
>epoch=463, lrate=0.500, error=0.000
>epoch=464, lrate=0.500, error=0.000
>epoch=465, lrate=0.500, error=0.000
>epoch=466, lrate=0.500, error=0.000
>epoch=467, lrate=0.500, error=0.000
>epoch=468, lrate=0.500, error=0.000
>epoch=469, lrate=0.500, error=0.000
>epoch=470, lrate=0.500, error=0.000
>epoch=471, lrate=0.500, error=0.000
>epoch=472, lrate=0.500, error=0.000
>epoch=473, lrate=0.500, error=0.000
>epoch=474, lrate=0.500, error=0.000
>epoch=475, lrate=0.500, error=0.000
>epoch=476, lrate=0.500, error=0.000
>

In [24]:
# Test making predictions with the network
for row in dataset:
    prediction = predict(network, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=38, Got=46
Expected=43, Got=9
Expected=33, Got=27
Expected=38, Got=27
Expected=38, Got=44
Expected=11, Got=12
Expected=38, Got=38
Expected=37, Got=27
Expected=39, Got=29
Expected=39, Got=8
Expected=38, Got=38
Expected=11, Got=9
Expected=43, Got=6
Expected=38, Got=27
Expected=18, Got=39
Expected=23, Got=8
Expected=43, Got=45
Expected=18, Got=14
Expected=23, Got=23
Expected=38, Got=38
Expected=38, Got=38
Expected=23, Got=9
Expected=39, Got=27
Expected=11, Got=27
Expected=6, Got=6
Expected=39, Got=45
Expected=39, Got=29
Expected=27, Got=27
Expected=0, Got=27
Expected=30, Got=30
Expected=0, Got=8
Expected=15, Got=27
Expected=30, Got=30
Expected=0, Got=46
Expected=29, Got=27
Expected=5, Got=29
Expected=0, Got=46
Expected=30, Got=21
Expected=27, Got=29
Expected=27, Got=27
Expected=29, Got=44
Expected=27, Got=44
Expected=32, Got=6
Expected=28, Got=29
Expected=4, Got=27
Expected=27, Got=29
Expected=29, Got=29
Expected=28, Got=21
Expected=8, Got=8
Expected=29, Got=29
Expected=34, Got=2

In [25]:
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

print('Scores: %s' % scores)

print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

IndexError: list assignment index out of range